# Importation of modules

In [3]:
from sage.all import (
    EllipticCurve,
    randint,
    ZZ,
    discrete_log,
    GF,
    matrix,
    block_matrix,
    set_random_seed
)
from sage.modules.free_module_integer import IntegerLattice

from Crypto.Hash import SHAKE256

import quaternion as quat
import endomorphism as End
import elliptic_curve as ec
import parameter_generate as param
import d2isogeny
import supersingular
import compression
import richelot_isogenies as richelot
import utilities_festa as utilities



In [6]:
import importlib
import sys
import time
import warnings  # Add this to handle PARI stack warnings

# Suppress PARI stack warnings (common in SageMath)
warnings.filterwarnings("ignore", category=RuntimeWarning, message="cypari2 leaked")

# Import your modules
import parameter_generate as param
import elliptic_curve as ec
import QFESTA
import utilities_festa as utilities

# Reload modules (only needed during development)
importlib.reload(ec)
importlib.reload(param)
importlib.reload(QFESTA)

if __name__ == "__main__":
    utilities.speed_up_sagemath()
    sys.setrecursionlimit(2000)

    N = 10
    print("Benchmarking QFESTA KEM")
    
    for model in ["Mumford", "Theta"]:
        print(f"\nUsing {model} model")
        use_theta = (model == "Theta")
        
        for lam in [128, 192, 256]:
            try:
                KEM = QFESTA.QFESTA_KEM(lam, use_theta=use_theta)
                t_gen = t_enc = t_dec = 0
                
                for _ in range(N):
                    # Key Generation
                    t = time.time()
                    sec_key, pub_key = KEM.Gen()  # Likely hangs here
                    t_gen += time.time() - t
                    
                    # Encapsulation
                    t = time.time()
                    K, ciphertext = KEM.Encaps(pub_key)
                    t_enc += time.time() - t
                    
                    # Decapsulation
                    t = time.time()
                    Kd = KEM.Decaps(ciphertext, sec_key, pub_key)
                    t_dec += time.time() - t
                    
                    assert K == Kd, "Decapsulation failed!"
                
                print(f"λ={lam}: Gen={t_gen/N:.2f}s, Enc={t_enc/N:.2f}s, Dec={t_dec/N:.2f}s")
                
            except KeyboardInterrupt:
                print(f"Interrupted at λ={lam}. Skipping...")
                break  # Exit if user interrupts
            except Exception as e:
                print(f"Error at λ={lam}: {str(e)}")

Benchmarking QFESTA KEM

Using Mumford model
λ=128: Gen=5.85s, Enc=6.99s, Dec=15.24s
λ=192: Gen=11.54s, Enc=13.69s, Dec=33.93s
λ=256: Gen=20.26s, Enc=24.94s, Dec=66.42s

Using Theta model
λ=128: Gen=2.49s, Enc=3.32s, Dec=8.82s
λ=192: Gen=5.82s, Enc=7.84s, Dec=23.52s
λ=256: Gen=9.76s, Enc=15.28s, Dec=48.58s


# Full benchmarking of QFESTA for Mumford and theta coordinates

In [8]:
import importlib
import sys
import time
import warnings
import pickle  # For size estimation via serialization

# Suppress PARI stack warnings (common in SageMath)
warnings.filterwarnings("ignore", category=RuntimeWarning, message="cypari2 leaked")

# Import your modules
import parameter_generate as param
import elliptic_curve as ec
import QFESTA
import utilities_festa as utilities

# Reload modules (only needed during development)
importlib.reload(ec)
importlib.reload(param)
importlib.reload(QFESTA)

def size_in_bytes(obj):
    try:
        return len(pickle.dumps(obj))
    except Exception:
        return sys.getsizeof(obj)

if __name__ == "__main__":
    utilities.speed_up_sagemath()
    sys.setrecursionlimit(2000)

    N = 10  # Number of trials per parameter set
    print("Benchmarking QFESTA KEM")
    
    for model in ["Mumford", "Theta"]:
        print(f"\nUsing {model} model")
        use_theta = (model == "Theta")
        
        for lam in [128, 192, 256]:
            try:
                KEM = QFESTA.QFESTA_KEM(lam, use_theta=use_theta)
                t_gen = t_enc = t_dec = 0

                sizes = {
                    "secret_key": 0,
                    "public_key": 0,
                    "ciphertext": 0
                }
                
                for _ in range(N):
                    # Key Generation
                    t = time.time()
                    sec_key, pub_key = KEM.Gen()
                    t_gen += time.time() - t

                    # Encapsulation
                    t = time.time()
                    K, ciphertext = KEM.Encaps(pub_key)
                    t_enc += time.time() - t

                    # Decapsulation
                    t = time.time()
                    Kd = KEM.Decaps(ciphertext, sec_key, pub_key)
                    t_dec += time.time() - t
                    
                    assert K == Kd, "Decapsulation failed!"

                    # Size calculations (only accumulate once)
                    sizes["secret_key"] += size_in_bytes(sec_key)
                    sizes["public_key"] += size_in_bytes(pub_key)
                    sizes["ciphertext"] += size_in_bytes(ciphertext)
                
                print(f"λ={lam}:")
                print(f"  Avg Gen Time = {t_gen/N:.2f}s")
                print(f"  Avg Enc Time = {t_enc/N:.2f}s")
                print(f"  Avg Dec Time = {t_dec/N:.2f}s")
                print(f"  Avg Secret Key Size  = {sizes['secret_key'] // N} bytes")
                print(f"  Avg Public Key Size  = {sizes['public_key'] // N} bytes")
                print(f"  Avg Ciphertext Size  = {sizes['ciphertext'] // N} bytes")

            except KeyboardInterrupt:
                print(f"Interrupted at λ={lam}. Skipping...")
                break
            except Exception as e:
                print(f"Error at λ={lam}: {str(e)}")


Benchmarking QFESTA KEM

Using Mumford model
λ=128:
  Avg Gen Time = 6.31s
  Avg Enc Time = 7.18s
  Avg Dec Time = 16.04s
  Avg Secret Key Size  = 2979 bytes
  Avg Public Key Size  = 262 bytes
  Avg Ciphertext Size  = 512 bytes
λ=192:
  Avg Gen Time = 12.56s
  Avg Enc Time = 14.59s
  Avg Dec Time = 36.92s
  Avg Secret Key Size  = 3860 bytes
  Avg Public Key Size  = 385 bytes
  Avg Ciphertext Size  = 752 bytes
λ=256:
  Avg Gen Time = 20.13s
  Avg Enc Time = 24.74s
  Avg Dec Time = 66.65s
  Avg Secret Key Size  = 4716 bytes
  Avg Public Key Size  = 505 bytes
  Avg Ciphertext Size  = 992 bytes

Using Theta model
λ=128:
  Avg Gen Time = 2.52s
  Avg Enc Time = 3.54s
  Avg Dec Time = 9.03s
  Avg Secret Key Size  = 2488 bytes
  Avg Public Key Size  = 262 bytes
  Avg Ciphertext Size  = 512 bytes
λ=192:
  Avg Gen Time = 5.53s
  Avg Enc Time = 7.78s
  Avg Dec Time = 22.84s
  Avg Secret Key Size  = 3130 bytes
  Avg Public Key Size  = 385 bytes
  Avg Ciphertext Size  = 752 bytes
λ=256:
  Avg Gen T